#Prerequisite
- Upload Folder Capital on root folder to google drive
- Upload this code to google colab 
- mount drive

In [ ]:
#works in colab not in local
!sudo apt install tesseract-ocr
!pip install pytesseract

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
from PIL import Image

import pytesseract
import cv2
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score


In [ ]:
# Fungsi untuk membaca dan memproses gambar, lalu mengenali teks
def recognize_text(image_path):
    # Membaca gambar
    image = cv2.imread(image_path)

    # Check if image is loaded successfully
    if image is None:
        print(f"Error: Could not load image at path: {image_path}")
        return ""  # Return empty string if image loading fails

    # Mengonversi gambar ke grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Mengasah tepi gambar
    sharpen_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharpen = cv2.filter2D(gray, -1, sharpen_kernel)

    # Menerapkan thresholding untuk mendapatkan gambar biner
    thresh = cv2.threshold(sharpen, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Opsional, Anda dapat menerapkan operasi morfologi untuk meningkatkan pengenalan teks
    kernel = np.ones((3, 3), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Menggunakan pytesseract untuk membaca teks dari gambar yang diproses
    recognized_text = pytesseract.image_to_string(morph)

    return recognized_text

In [ ]:
# Fungsi untuk membaca ground truth dari file CSV
def read_ground_truth(csv_file):
    ground_truth_dict = {}
    with open(csv_file, mode='r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Lewati header CSV
        for row in csv_reader:
            image_name = row[0]
            ground_truth = row[1]
            ground_truth_dict[image_name] = ground_truth
    return ground_truth_dict

In [ ]:
# Fungsi untuk menghitung akurasi per karakter
def calculate_char_accuracy(recognized_text, ground_truth):
    recognized_text = recognized_text.upper()
    ground_truth = ground_truth.upper()

    TP = sum(1 for i in range(min(len(recognized_text), len(ground_truth))) if recognized_text[i] == ground_truth[i])
    FP = sum(1 for i in range(len(recognized_text)) if i >= len(ground_truth) or recognized_text[i] != ground_truth[i])
    FN = sum(1 for i in range(len(ground_truth)) if i >= len(recognized_text) or recognized_text[i] != ground_truth[i])

    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    # Check if the denominator is zero before calculating accuracy
    denominator = max(len(recognized_text), len(ground_truth))
    accuracy = (TP / denominator) * 100 if denominator != 0 else 0  # Assign 0 if denominator is 0

    return accuracy, f1, TP, FP, FN

In [ ]:
# Fungsi untuk memplot confusion matrix
def plot_confusion_matrix(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    plt.figure(figsize=(10, 7))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix tesseract')
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.ylabel('True Labels')
    plt.xlabel('Predicted Labels')

    for i in range(len(labels)):
        for j in range(len(labels)):
            plt.text(j, i, cm[i, j], ha='center', va='center', color='red')

    plt.tight_layout()
    plt.show()

In [ ]:
# Fungsi untuk memplot grafik akurasi
def plot_accuracy_graph(accuracies, ground_truth_lengths):
    plt.figure(figsize=(10, 6))
    plt.plot(ground_truth_lengths, accuracies, marker='o', linestyle='-', color='b', label='Akurasi per Gambar')
    plt.xlabel('Panjang String Ground Truth')
    plt.ylabel('Akurasi (%)')
    plt.title('Akurasi tesseract')
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
def plot_accuracy_vs_iteration(accuracies):
    # Menghitung rata-rata kumulatif
    avg_accuracies = [sum(accuracies[:i + 1]) / (i + 1) for i in range(len(accuracies))]

    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(accuracies) + 1), accuracies, marker='o', linestyle='-', color='g', label='Akurasi per Iterasi')
    plt.plot(range(1, len(avg_accuracies) + 1), avg_accuracies, marker='', linestyle='--', color='b', label='Rata-rata Kumulatif')
    plt.xlabel('Nomor Iterasi')
    plt.ylabel('Akurasi (%)')
    plt.title('Akurasi pyTesseract 30 Char dengan Rata-rata Kumulatif')
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
def plot_f1_graph(f1_scores):
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(f1_scores) + 1), f1_scores, marker='o', linestyle='-', color='r', label='F1 Score per Gambar')
    plt.xlabel('Nomor Gambar')
    plt.ylabel('F1 Score')
    plt.title('F1 Convidence tesseract')
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
# Membaca ground truth dari file CSV
ground_truth_file = 'drive/MyDrive/Capital/GROUND_TRUTH.csv'  # Ganti dengan path file CSV sesuai dengan path gdrive Anda
ground_truth_dict = read_ground_truth(ground_truth_file)

# Variabel untuk menghitung confusion matrix dan F1 score total
all_true = []
all_pred = []
f1_scores = []

# Variabel untuk menyimpan akurasi per gambar
accuracies = []
ground_truth_lengths = []  # Menyimpan panjang karakter ground truth

In [ ]:
# Loop melalui gambar TEST_1.jpg hingga TEST_19.jpg dan bandingkan hasilnya
for i in range(1, 36):
    # Membuat path file gambar
    image_path = f'drive/MyDrive/Capital/TEST_{i}.png' # Ganti dengan path gambar sesuai dengan path gdrive 

    # Memanggil fungsi recognize_text
    recognized_text = recognize_text(image_path)

    # Mencari ground truth untuk gambar ini
    image_name = f'TEST_{i}.png'
    ground_truth = ground_truth_dict.get(image_name, "")

    # Menghitung akurasi karakter, F1 score, dan confusion matrix
    accuracy, f1, TP, FP, FN = calculate_char_accuracy(recognized_text, ground_truth)

    f1_scores.append(f1)

    # Menyimpan nilai untuk confusion matrix global
    all_true.extend(list(ground_truth.upper()))  # Menyimpan ground truth
    if recognized_text:  # Pastikan recognized_text tidak kosong
        all_pred.extend(list(recognized_text.upper()))  # Menyimpan hasil pengenalan
    else:
        all_pred.extend([''] * len(ground_truth))  # Menambahkan karakter kosong jika tidak ada teks yang dikenali

    # Menyimpan akurasi per gambar dan panjang karakter ground truth
    accuracies.append(accuracy)
    ground_truth_lengths.append(len(ground_truth))  # Menyimpan panjang karakter ground truth

    # Menampilkan hasil per gambar
    print(f"Gambar {i} - Akurasi: {accuracy:.2f}% - F1 Score: {f1:.2f} - TP: {TP} - FP: {FP} - FN: {FN}")

# Menyesuaikan panjang all_pred agar sesuai dengan all_true (padding jika perlu)
if len(all_true) > len(all_pred):
    all_pred.extend([''] * (len(all_true) - len(all_pred)))  # Padding untuk all_pred
elif len(all_true) < len(all_pred):
    all_true.extend([''] * (len(all_pred) - len(all_true)))  # Padding untuk all_true

# Menghitung confusion matrix secara keseluruhan
# Menghitung confusion matrix secara keseluruhan
label = sorted(set(all_pred))  # This line can be removed or repurposed if needed
conf_matrix = confusion_matrix(all_true, all_pred, labels=label)  # Change here: label to labels=label
f1_total = f1_score(all_true, all_pred, average='weighted')

# Menampilkan confusion matrix dan F1 score total
print("\nConfusion Matrix:")
print(conf_matrix)
print(f"Total F1 Score (Weighted): {f1_total:.2f}")

labels = sorted(set(all_pred))




In [ ]:
plot_confusion_matrix(all_true, all_pred, labels)


In [ ]:
plot_accuracy_graph(accuracies, ground_truth_lengths)

In [ ]:
plot_f1_graph(f1_scores)
